## Multi-Method 3D Blur Processing: Gaussian, Mean, and Median for TIFF Files 

In [1]:
import os
import cv2
import numpy as np
from PIL import Image, ImageSequence, UnidentifiedImageError
from ipywidgets import widgets
from tkinter import Tk, filedialog, messagebox
from scipy.ndimage import gaussian_filter, median_filter

# Function to select a file using tkinter
def select_file(prompt="Select .tif File"):
    """
    Opens a file dialog using tkinter to allow the user to select a single .tif file.

    Args:
        prompt (str): The prompt that will be displayed in the file selection dialog. Defaults to "Select .tif File".

    Returns:
        str: The file path of the selected .tif file, or an empty string if no file is selected.
    """
    root = Tk()
    root.withdraw()  # Hide the root window
    file = filedialog.askopenfilename(title=prompt, filetypes=[("TIFF files", "*.tif")])
    root.update()
    return file

# Function to select directory using tkinter
def select_directory(prompt="Select Directory"):
    """
    Opens a directory selection dialog using tkinter to allow the user to select a folder.

    Args:
        prompt (str): The prompt that will be displayed in the directory selection dialog. Defaults to "Select Directory".

    Returns:
        str: The path of the selected directory, or an empty string if no directory is selected.
    """
    root = Tk()
    root.withdraw()  # Hide the root window
    folder = filedialog.askdirectory(title=prompt)
    root.update()
    return folder

# Apply 3D Gaussian blur to a 3D image with different radii for x, y, and z axes
def apply_3d_gaussian_blur(image_array, blur_radius_x, blur_radius_y, blur_radius_z):
    """
    Applies a 3D Gaussian blur to the input image array with different blur radii for the x, y, and z axes.

    Args:
        image_array (numpy.ndarray): The 3D image data as a numpy array.
        blur_radius_x (float): The Gaussian blur radius along the x-axis.
        blur_radius_y (float): The Gaussian blur radius along the y-axis.
        blur_radius_z (float): The Gaussian blur radius along the z-axis.

    Returns:
        numpy.ndarray: The blurred image array, with values clipped to the range [0, 255] and converted to uint8 format.
    """
    # Handle anisotropic voxel blur, scale different axes as per voxel sizes
    blurred = gaussian_filter(image_array, sigma=(blur_radius_z, blur_radius_y, blur_radius_x))
    return np.clip(blurred, 0, 255).astype(np.uint8)

# Apply 3D Median blur
def apply_3d_median_blur(image_array, kernel_size_x, kernel_size_y, kernel_size_z):
    """
    Applies a 3D median blur to the input image array using a specified kernel size for the x, y, and z axes.

    Args:
        image_array (numpy.ndarray): The 3D image data as a numpy array.
        kernel_size_x (int): The size of the median filter kernel along the x-axis.
        kernel_size_y (int): The size of the median filter kernel along the y-axis.
        kernel_size_z (int): The size of the median filter kernel along the z-axis.

    Returns:
        numpy.ndarray: The blurred image array, with values clipped to the range [0, 255] and converted to uint8 format.
    """
    blurred = median_filter(image_array, size=(kernel_size_z, kernel_size_y, kernel_size_x))
    return np.clip(blurred, 0, 255).astype(np.uint8)

# Apply 3D Mean blur
def apply_3d_mean_blur(image_array, kernel_size_x, kernel_size_y, kernel_size_z):
    """
    Applies a 3D mean blur to the input image array using a specified kernel size for the x, y, and z axes.

    Args:
        image_array (numpy.ndarray): The 3D image data as a numpy array.
        kernel_size_x (int): The size of the mean filter kernel along the x-axis.
        kernel_size_y (int): The size of the mean filter kernel along the y-axis.
        kernel_size_z (int): The size of the mean filter kernel along the z-axis (unused in this function).

    Returns:
        numpy.ndarray: The blurred image array, with values clipped to the range [0, 255] and converted to uint8 format.
    """
    blurred = cv2.blur(image_array, (kernel_size_x, kernel_size_y))
    return np.clip(blurred, 0, 255).astype(np.uint8)
    
# Save processed images (frames) as a multi-page TIFF
def save_image(frames, output_dir, file_name, suffix):
    """
    Saves a series of frames as a multi-page TIFF file.

    Args:
        frames (list of PIL.Image): List of PIL Image objects representing individual frames of the 3D image.
        output_dir (str): The directory where the output TIFF file will be saved.
        file_name (str): The base name of the output file, without extension.
        suffix (str): A suffix to append to the file name, indicating the type of blur applied.

    Returns:
        None
    """
    output_path = os.path.join(output_dir, f"{file_name}_{suffix}.tif")
    frames[0].save(output_path, save_all=True, append_images=frames[1:])
    print(f"Saved multi-page TIFF: {output_path}")

# Process 3D TIFF file for blurring with enhanced error handling
def process_file_blur(file_path, output_dir, blur_type, blur_radius_x=0, blur_radius_y=0, blur_radius_z=0, kernel_size=3):
    """
    Processes a 3D TIFF file by applying a specified blur type (Gaussian, Mean, Median) and saves the result.

    Args:
        file_path (str): The path to the input .tif file.
        output_dir (str): The directory where the output blurred TIFF will be saved.
        blur_type (str): The type of blur to apply ("Gaussian", "Mean", or "Median").
        blur_radius_x (float, optional): The blur radius along the x-axis for Gaussian blur. Defaults to 0.
        blur_radius_y (float, optional): The blur radius along the y-axis for Gaussian blur. Defaults to 0.
        blur_radius_z (float, optional): The blur radius along the z-axis for Gaussian blur. Defaults to 0.
        kernel_size (int, optional): The kernel size for Median and Mean blur. Defaults to 3.

    Returns:
        None
    """
    # Check if the file has a .tif extension
    if not file_path.lower().endswith('.tif'):
        messagebox.showerror("Invalid File", "Selected file is not a .tif file.")
        print(f"Skipping non-.tif file: {file_path}")
        return

    try:
        with Image.open(file_path) as img:
            if img.format != 'TIFF':
                messagebox.showerror("Invalid File", "Selected file is not a valid TIFF image.")
                print(f"File {file_path} is not a valid TIFF image. Skipping.")
                return
            
            # Load 3D image (stack of frames)
            frames = [np.array(frame) for frame in ImageSequence.Iterator(img)]
            image_3d_array = np.stack(frames)  # Stack frames into a 3D array (Z, Y, X)
            print(f"Loaded 3D image with shape: {image_3d_array.shape}")

            # Apply selected blur method
            if blur_type == "Gaussian":
                blurred_image_3d = apply_3d_gaussian_blur(image_3d_array, blur_radius_x, blur_radius_y, blur_radius_z)
                print(f"Applied Gaussian blur with sigma_x={blur_radius_x}, sigma_y={blur_radius_y}, sigma_z={blur_radius_z}")
            elif blur_type == "Median":
                blurred_image_3d = apply_3d_median_blur(image_3d_array, kernel_size, kernel_size, kernel_size_z=kernel_size)
                print(f"Applied Median blur with kernel size={kernel_size}")
            elif blur_type == "Mean":
                blurred_image_3d = apply_3d_mean_blur(image_3d_array, kernel_size, kernel_size, kernel_size)
                print(f"Applied Mean blur with kernel size={kernel_size}")

            # Convert back to PIL Image sequence
            blurred_frames = [Image.fromarray(blurred_image_3d[i, :, :]) for i in range(blurred_image_3d.shape[0])]
            
            # Save blurred 3D TIFF image
            file_name = os.path.splitext(os.path.basename(file_path))[0]
            save_image(blurred_frames, output_dir, file_name, f"{blur_type.lower()}_blur")
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
    except PermissionError:
        print(f"Error: Permission denied for file {file_path}.")
    except MemoryError:
        print("Error: Not enough memory to process the image.")
    except UnidentifiedImageError:
        print(f"Error: The file {file_path} is not a valid image.")
    except IOError as e:
        print(f"File {file_path} cannot be opened: {e}")
    except Exception as e:
        print(f"Unexpected error processing file {file_path}: {e}")

# Select input/output directories and set blur parameters
input_file = ""
output_dir = ""

# Function to select input file and output directory
def select_file_and_dir():
    """
    Allows the user to select a TIFF file and an output directory using a graphical file and directory selection dialog.

    Returns:
        None
    """
    global input_file, output_dir
    input_file = select_file("Select .tif File")
    if input_file:
        output_dir = select_directory("Select Output Directory")
        print(f"Input File: {input_file}")
        print(f"Output Directory: {output_dir}")

# Function to start blurring process
def start_blurring(blur_type, blur_radius_x, blur_radius_y, blur_radius_z, kernel_size):
    """
    Starts the blurring process by applying the selected blur type (Gaussian, Mean, Median) on the selected file.

    Args:
        blur_type (str): The type of blur to apply ("Gaussian", "Mean", or "Median").
        blur_radius_x (float): The blur radius along the x-axis (for Gaussian blur).
        blur_radius_y (float): The blur radius along the y-axis (for Gaussian blur).
        blur_radius_z (float): The blur radius along the z-axis (for Gaussian blur).
        kernel_size (int): The size of the kernel (for Mean and Median blurs).

    Returns:
        None
    """
    if input_file and output_dir:
        process_file_blur(input_file, output_dir, blur_type, blur_radius_x, blur_radius_y, blur_radius_z, kernel_size)
        print("Blurring completed.")
    else:
        print("Please select input file and output directory first.")

# Widget for selecting file and directories
file_dir_selector = widgets.Button(description="Select File and Directory")
file_dir_selector.on_click(lambda x: select_file_and_dir())

# Widgets for blurring
blur_type_dropdown = widgets.Dropdown(
    options=["Gaussian", "Mean", "Median"],
    value="Gaussian",
    description="Blur Type:",
)

blur_x_input = widgets.IntText(value=3, description="Blur Radius X")
blur_y_input = widgets.IntText(value=3, description="Blur Radius Y")
blur_z_input = widgets.IntText(value=1, description="Blur Radius Z")  # Z-axis control

# Initially hide Kernel Size input
kernel_size_input = widgets.IntText(value=3, description="Kernel Size", layout=widgets.Layout(display='none'))
blur_button = widgets.Button(description="Start Blurring")

# Function to lock/unlock kernel size based on blur type and hide/show it
def update_kernel_size_visibility(change):
    """
    Updates the visibility of the kernel size input widget based on the selected blur type.

    If "Gaussian" is selected in the `blur_type_dropdown`, the kernel size input is hidden.
    If "Mean" or "Median" is selected, the kernel size input is shown.

    Args:
        change (dict): Contains information about the change event, specifically the new value of the dropdown selection.
    """
    if change['new'] == "Gaussian":
        kernel_size_input.layout.display = 'none'  # Hide kernel size for Gaussian
    else:
        kernel_size_input.layout.display = 'block'  # Show kernel size for Mean/Median

# Add observer to blur_type_dropdown to monitor changes
blur_type_dropdown.observe(update_kernel_size_visibility, names='value')

# Set the blurring function to run when the blur button is clicked
blur_button.on_click(lambda x: start_blurring(
    blur_type_dropdown.value,
    blur_x_input.value if blur_type_dropdown.value == "Gaussian" else 0,
    blur_y_input.value if blur_type_dropdown.value == "Gaussian" else 0,
    blur_z_input.value if blur_type_dropdown.value == "Gaussian" else 0,
    kernel_size_input.value if blur_type_dropdown.value in ["Mean", "Median"] else 3
))

# Display widgets
display(file_dir_selector, blur_type_dropdown, blur_x_input, blur_y_input, blur_z_input, kernel_size_input, blur_button)

Button(description='Select File and Directory', style=ButtonStyle())

Dropdown(description='Blur Type:', options=('Gaussian', 'Mean', 'Median'), value='Gaussian')

IntText(value=3, description='Blur Radius X')

IntText(value=3, description='Blur Radius Y')

IntText(value=1, description='Blur Radius Z')

IntText(value=3, description='Kernel Size', layout=Layout(display='none'))

Button(description='Start Blurring', style=ButtonStyle())

In [ ]:
# Function to select a file using tkinter
def select_file(prompt="Select .tif File"):
    root = Tk()
    root.withdraw()  # Hide the root window
    file = filedialog.askopenfilename(title=prompt, filetypes=[("TIFF files", "*.tif")])
    root.update()
    return file


# Function to select directory using tkinter
def select_directory(prompt="Select Directory"):
    root = Tk()
    root.withdraw()  # Hide the root window
    folder = filedialog.askdirectory(title=prompt)
    root.update()
    return folder


# Apply 3D Gaussian blur to a 3D image with different radii for x, y, and z axes
def apply_3d_gaussian_blur(image_array, blur_radius_x, blur_radius_y, blur_radius_z):
    blurred = gaussian_filter(image_array, sigma=(blur_radius_z, blur_radius_y, blur_radius_x))
    return np.clip(blurred, 0, 255).astype(np.uint8)


# Apply 3D Median blur
def apply_3d_median_blur(image_array, kernel_size_x, kernel_size_y, kernel_size_z):
    blurred = median_filter(image_array, size=(kernel_size_z, kernel_size_y, kernel_size_x))
    return np.clip(blurred, 0, 255).astype(np.uint8)


# Apply 3D Mean blur
def apply_3d_mean_blur(image_array, kernel_size_x, kernel_size_y, kernel_size_z):
    blurred = cv2.blur(image_array, (kernel_size_x, kernel_size_y))
    return np.clip(blurred, 0, 255).astype(np.uint8)


# Save processed images (frames) as a multi-page TIFF
def save_image(frames, output_dir, file_name, suffix):
    output_path = os.path.join(output_dir, f"{file_name}_{suffix}.tif")
    frames[0].save(output_path, save_all=True, append_images=frames[1:])
    print(f"Saved multi-page TIFF: {output_path}")


# Process 3D TIFF file for blurring with enhanced error handling
def process_file_blur(file_path, output_dir, blur_type, blur_radius_x=0, blur_radius_y=0, blur_radius_z=0, kernel_size=3):
    # Check if the file has a .tif extension
    if not file_path.lower().endswith('.tif'):
        messagebox.showerror("Invalid File", "Selected file is not a .tif file.")
        print(f"Skipping non-.tif file: {file_path}")
        return

    try:
        with Image.open(file_path) as img:
            if img.format != 'TIFF':
                messagebox.showerror("Invalid File", "Selected file is not a valid TIFF image.")
                print(f"File {file_path} is not a valid TIFF image. Skipping.")
                return
            
            # Load 3D image (stack of frames)
            frames = [np.array(frame) for frame in ImageSequence.Iterator(img)]
            image_3d_array = np.stack(frames)  # Stack frames into a 3D array (Z, Y, X)
            print(f"Loaded 3D image with shape: {image_3d_array.shape}")

            # Apply selected blur method
            if blur_type == "Gaussian":
                blurred_image_3d = apply_3d_gaussian_blur(image_3d_array, blur_radius_x, blur_radius_y, blur_radius_z)
                print(f"Applied Gaussian blur with sigma_x={blur_radius_x}, sigma_y={blur_radius_y}, sigma_z={blur_radius_z}")
            elif blur_type == "Median":
                blurred_image_3d = apply_3d_median_blur(image_3d_array, kernel_size, kernel_size, kernel_size_z=kernel_size)
                print(f"Applied Median blur with kernel size={kernel_size}")
            elif blur_type == "Mean":
                blurred_image_3d = apply_3d_mean_blur(image_3d_array, kernel_size, kernel_size, kernel_size)
                print(f"Applied Mean blur with kernel size={kernel_size}")

            # Convert back to PIL Image sequence
            blurred_frames = [Image.fromarray(blurred_image_3d[i, :, :]) for i in range(blurred_image_3d.shape[0])]
            
            # Save blurred 3D TIFF image
            file_name = os.path.splitext(os.path.basename(file_path))[0]
            save_image(blurred_frames, output_dir, file_name, f"{blur_type.lower()}_blur")
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
    except PermissionError:
        print(f"Error: Permission denied for file {file_path}.")
    except MemoryError:
        print("Error: Not enough memory to process the image.")
    except UnidentifiedImageError:
        print(f"Error: The file {file_path} is not a valid image.")
    except IOError as e:
        print(f"File {file_path} cannot be opened: {e}")
    except Exception as e:
        print(f"Unexpected error processing file {file_path}: {e}")

# Select input/output directories and set blur parameters
def select_file_and_dir():
    global input_file, output_dir
    input_file = select_file("Select .tif File")
    if input_file:
        output_dir = select_directory("Select Output Directory")
        print(f"Input File: {input_file}")
        print(f"Output Directory: {output_dir}")


# Function to start blurring process
def start_blurring(blur_type, blur_radius_x, blur_radius_y, blur_radius_z, kernel_size):
    if input_file and output_dir:
        process_file_blur(input_file, output_dir, blur_type, blur_radius_x, blur_radius_y, blur_radius_z, kernel_size)
        print("Blurring completed.")
    else:
        print("Please select input file and output directory first.")
        
# Widget for selecting file and directories
file_dir_selector = widgets.Button(description="Select File and Directory")
file_dir_selector.on_click(lambda x: select_file_and_dir())

# Widgets for blurring
blur_type_dropdown = widgets.Dropdown(
    options=["Gaussian", "Mean", "Median"],
    value="Gaussian",
    description="Blur Type:",
)

blur_x_input = widgets.IntText(value=3, description="Blur Radius X")
blur_y_input = widgets.IntText(value=3, description="Blur Radius Y")
blur_z_input = widgets.IntText(value=1, description="Blur Radius Z")  # Z-axis control

# Initially hide Kernel Size input
kernel_size_input = widgets.IntText(value=3, description="Kernel Size", layout=widgets.Layout(display='none'))

blur_button = widgets.Button(description="Start Blurring")

# Function to lock/unlock kernel size based on blur type and hide/show it
def update_kernel_size_visibility(change):
    if change['new'] == "Gaussian":
        kernel_size_input.layout.display = 'none'  # Hide kernel size for Gaussian
    else:
        kernel_size_input.layout.display = 'block'  # Show kernel size for Mean/Median

# Add observer to blur_type_dropdown to monitor changes
blur_type_dropdown.observe(update_kernel_size_visibility, names='value')

# Set the blurring function to run when the blur button is clicked
blur_button.on_click(lambda x: start_blurring(
    blur_type_dropdown.value,
    blur_x_input.value if blur_type_dropdown.value == "Gaussian" else 0,
    blur_y_input.value if blur_type_dropdown.value == "Gaussian" else 0,
    blur_z_input.value if blur_type_dropdown.value == "Gaussian" else 0,
    kernel_size_input.value if blur_type_dropdown.value in ["Mean", "Median"] else 3
))

# Display widgets
display(file_dir_selector, blur_type_dropdown, blur_x_input, blur_y_input, blur_z_input, kernel_size_input, blur_button)